## Instalação das libs

In [ ]:
!pip install -U spacy -q

# Baixa o modelo de linguagem em português
!python -m spacy download pt_core_news_lg -q

# Instala a biblioteca gdown para downloads do Google Drive
!pip install -q gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install ipywidgets==7.7.2 # funcao especifica para funcionar no colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 25.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.15
    Uninstalling jupyterlab_widgets-3.0.15:
      Successfully uninstalled jupyterlab_widgets-3.0.15
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import gdown
import spacy
from collections import defaultdict, Counter
import re
import numpy as np
import math
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Faz o download
gdown.download(f"https://drive.google.com/uc?id=1tTcsitS5NUFku9do_PP8eQ-TPw6_Lh4t&export=download", "Livro-O-Guarani.txt", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1tTcsitS5NUFku9do_PP8eQ-TPw6_Lh4t&export=download
To: /content/Livro-O-Guarani.txt
100%|██████████| 654k/654k [00:00<00:00, 131MB/s]


'Livro-O-Guarani.txt'

In [ ]:
# --- FUNÇÕES AUXILIARES ---
def readFile():
    try:
      with open("Livro-O-Guarani.txt", "r", encoding="utf-8") as f:
          full_text = f.read()
      print("livro carregado.")
      return full_text
    except FileNotFoundError:
        print("arquivo nao encontrado")
        return None
    except Exception as e:
        print(f"erro: {e}")
        return None

def getTextInit(full_text):
    text_lines = full_text.splitlines()
    prologue_line_index = -1
    for i, line in enumerate(text_lines):
        if "PRÓLOGO" in line and line.strip() == "PRÓLOGO":
            prologue_line_index = i
            break
    if prologue_line_index != -1:
        text = "\n".join(text_lines[prologue_line_index:])
        print("cabecalho removido")
        return text
    else:
        print("'PRÓLOGO' não encontrado.")
        return full_text

In [ ]:
# --- CLASSES DO PIPELINE ---
class Chunker:
    def __init__(self, chunk_size=300, overlap=150):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def preprocess(self, text):
        text = text.lower()
        text = re.sub(r'\r\n|\r', '\n', text)
        text = re.sub(r'\n{2,}', '\n', text)
        text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'[“”‘’]', '"', text)
        text = re.sub(r'[–—]', '—', text)
        text = re.sub(r'\s*—\s*', ' — ', text)
        text = re.sub(r'…', '...', text)
        text = re.sub(r'\.\.\.\s*', '... ', text)
        text = re.sub(r'\s*;\s*', '; ', text)
        text = re.sub(r'\s*:\s*', ': ', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def create_chunks(self, text):
        words = text.split()
        chunks = []
        for i in range(0, len(words), self.chunk_size - self.overlap):
            chunk = ' '.join(words[i:i + self.chunk_size])
            chunks.append(chunk)
        return chunks

# ============================================================

class Analyser:
    def __init__(self, lang_model='pt_core_news_lg'):
        self.nlp = spacy.load(lang_model)

    def analyse(self, chunk):
        doc = self.nlp(chunk)
        persons = [ent.text for ent in doc.ents if ent.label_ == 'PER']
        places = [ent.text for ent in doc.ents if ent.label_ in ['LOC', 'GPE']]
        actions = [token.lemma_ for token in doc if token.pos_ == 'VERB' and not token.is_stop]
        processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
        entity_weights = self._calculate_weights(doc, persons + places + actions)
        return {"persons": persons, "places": places, "actions": actions, "processed_text": processed_text, "entity_weights": entity_weights}

    def _calculate_weights(self, doc, entities):
        weights = {}
        length = len(doc)
        for ent in entities:
            freq = doc.text.count(ent)
            positions = [1.0 - abs(t.i - length / 2) / (length / 2) for t in doc if ent.lower() in t.text.lower()]
            position_weight = np.mean(positions) if positions else 0
            weights[ent] = freq * 0.7 + position_weight * 0.3
        return weights

# ============================================================

class Indexer:
    def __init__(self):
        self.index = defaultdict(list)

    def build_index(self, chunk_metadata):
        for i, meta in enumerate(chunk_metadata):
            for p in meta['persons']: self.index[f'person:{p.lower()}'].append(i)
            for pl in meta['places']: self.index[f'place:{pl.lower()}'].append(i)
            for a in meta['actions']: self.index[f'action:{a.lower()}'].append(i)

    def query(self, intent):
        candidates = set()
        for p in intent['persons']: candidates.update(self.index.get(f'person:{p.lower()}', []))
        for pl in intent['places']: candidates.update(self.index.get(f'place:{pl.lower()}', []))
        for a in intent['actions']: candidates.update(self.index.get(f'action:{a.lower()}', []))
        return list(candidates)

# ============================================================

class Searcher:
    def __init__(self, max_features=1000):
        self.max_features = max_features
        self.vocabulary = []
        self.idf = {}
        self.doc_tfidf = None
        self.vectors = None

    def fit(self, texts):
        df = defaultdict(int)
        tokenized = [text.lower().split() for text in texts]
        for tokens in tokenized:
            for term in set(tokens):
                df[term] += 1
        most_common = sorted(df.items(), key=lambda x: x[1], reverse=True)
        self.vocabulary = [term for term, _ in most_common[:self.max_features]]
        N = len(texts)
        self.idf = {term: math.log((N + 1) / (df.get(term, 0) + 1)) + 1 for term in self.vocabulary}
        docs = []
        for tokens in tokenized:
            tf = Counter(tokens)
            vec = np.zeros(len(self.vocabulary), dtype=float)
            for i, term in enumerate(self.vocabulary):
                tf_val = tf[term] / len(tokens) if tokens else 0.0
                vec[i] = tf_val * self.idf[term]
            norm = np.linalg.norm(vec)
            if norm > 0: vec /= norm
            docs.append(vec)
        self.doc_tfidf = np.vstack(docs)
        self.vectors = self.doc_tfidf

    def query_vector(self, query_text):
        tokens = query_text.lower().split()
        tf = Counter(tokens)
        vec = np.zeros(len(self.vocabulary), dtype=float)
        for i, term in enumerate(self.vocabulary):
            tf_val = tf[term] / len(tokens) if tokens else 0.0
            vec[i] = tf_val * self.idf.get(term, 0.0)
        norm = np.linalg.norm(vec)
        if norm > 0: vec /= norm
        return vec.reshape(1, -1)

    def _cosine(self, vec1, vec2):
        return float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))

    def search(self, query_vec, candidate_ids, metadata, intent, boost_func, top_k=3):
        results = []
        q = query_vec.flatten()
        for idx in candidate_ids:
            doc_vec = self.vectors[idx].flatten()
            base_sim = self._cosine(q, doc_vec)
            boost = boost_func(idx, intent)
            final_score = base_sim * 0.7 + boost * 0.3
            results.append((idx, final_score))
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]

# ============================================================

class ChatbotPipeline:
    def __init__(self):
        self.chunker = Chunker()
        self.analyzer = Analyser()
        self.indexer = Indexer()
        self.searcher = Searcher()
        self.chunks = []
        self.chunk_metadata = []

    def process_text(self, text):
        clean = self.chunker.preprocess(text)
        self.chunks = self.chunker.create_chunks(clean)
        for i, chunk in enumerate(self.chunks):
            data = self.analyzer.analyse(chunk)
            self.chunk_metadata.append({"id": i, "original_text": chunk, **data})
        texts = [meta['processed_text'] for meta in self.chunk_metadata]
        self.searcher.fit(texts)
        self.indexer.build_index(self.chunk_metadata)

    def analyze_intent(self, query):
        doc = self.analyzer.nlp(query)
        persons = [ent.text for ent in doc.ents if ent.label_ == 'PER']
        places  = [ent.text for ent in doc.ents if ent.label_ in ['LOC','GPE']]
        actions = [token.lemma_ for token in doc if token.pos_ == 'VERB']

        q = query.lower()
        if any(k in q for k in ('resuma','resumir','resumo')):
            qt = 'summary'
        elif any(k in q for k in ('interpreta','interpretação','significado')):
            qt = 'literary'
        elif 'quem' in q:
            qt = 'person'
        elif 'onde' in q:
            qt = 'place'
        elif 'quando' in q:
            qt = 'time'
        elif 'como' in q:
            qt = 'method'
        elif 'por que' in q or 'porque' in q:
            qt = 'reason'
        else:
            qt = 'general'

        return {
            'persons': persons,
            'places': places,
            'actions': actions,
            'question_type': qt
        }


    def calculate_context_boost(self, chunk_id, intent):
        meta = self.chunk_metadata[chunk_id]
        score = 0
        for p in intent['persons']:
            if p.lower() in [x.lower() for x in meta['persons']]: score += meta['entity_weights'].get(p, 0) * 0.3
        for l in intent['places']:
            if l.lower() in [x.lower() for x in meta['places']]: score += meta['entity_weights'].get(l, 0) * 0.3
        for a in intent['actions']:
            if a.lower() in [x.lower() for x in meta['actions']]: score += meta['entity_weights'].get(a, 0) * 0.2
        return min(score, 1.0)

    def search(self, query, top_k=3):
        intent = self.analyze_intent(query)
        query_doc = self.analyzer.nlp(query)
        query_text = ' '.join([t.lemma_.lower() for t in query_doc if not t.is_stop and not t.is_punct])
        qvec = self.searcher.query_vector(query_text)
        candidates = self.indexer.query(intent)
        if not candidates: candidates = list(range(len(self.chunks)))
        ranked = self.searcher.search(qvec, candidates, self.chunk_metadata, intent, self.calculate_context_boost, top_k)
        return [{"chunk_id": i, "text": self.chunks[i], "score": score, "metadata": self.chunk_metadata[i]} for i, score in ranked]

In [ ]:
livro_completo = readFile()
livro_texto = getTextInit(livro_completo)

print("\n--- INÍCIO DO LIVRO (AMOSTRA) ---")
print(livro_texto[:500] + "...")

livro carregado.
cabecalho removido

--- INÍCIO DO LIVRO (AMOSTRA) ---
                                                 PRÓLOGO


Minha prima. — Gostou da minha história, e pede-me um romance; acha que posso fazer
alguma coisa neste ramo de literatura.
Engana-se; quando se conta aquilo que nos impressionou profundamente, o coração é que
fala; quando se exprime aquilo que outros sentiram ou podem sentir, fala a memória ou a
imaginação.
Esta pode errar, pode exagerar-se; o coração é sempre verdadeiro, não diz senão o que sentiu;
e o sentimento, qualquer que ele seja,...


In [ ]:
# pre-processamento
chunker_temp = Chunker()
texto_preprocessado = chunker_temp.preprocess(livro_texto)

print("\n--- AMOSTRA DO TEXTO PRÉ-PROCESSADO ---")
print(texto_preprocessado[:500] + "...")


--- AMOSTRA DO TEXTO PRÉ-PROCESSADO ---
prólogo minha prima gostou da minha história e pedeme um romance acha que posso fazer alguma coisa neste ramo de literatura enganase quando se conta aquilo que nos impressionou profundamente o coração é que fala quando se exprime aquilo que outros sentiram ou podem sentir fala a memória ou a imaginação esta pode errar pode exagerarse o coração é sempre verdadeiro não diz senão o que sentiu e o sentimento qualquer que ele seja tem a sua beleza assim não me julgo habilitado a escrever um romance a...


In [ ]:
# pipeline para processar texto
chatbot = ChatbotPipeline()
chatbot.process_text(livro_texto)

print(f"\nO texto foi dividido em {len(chatbot.chunks)} chunks.")

# cria um dataframe do pandas para visualizar a matriz TF-IDF
# mostra apenas os 5 primeiros chunks (linhas) e os 10 primeiros termos do vocabulario (colunas)
df_tfidf = pd.DataFrame(
    chatbot.searcher.doc_tfidf[:5, :10],
    index=[f'Chunk_{i}' for i in range(5)],
    columns=chatbot.searcher.vocabulary[:10]
)

print("\n--- MATRIZ TERMO-DOCUMENTO - AMOSTRA ---")
display(df_tfidf)


O texto foi dividido em 711 chunks.

--- MATRIZ TERMO-DOCUMENTO - AMOSTRA ---


,ter,haver,peri,cecília,de,d,índio,dizer,passar,olho
Chunk_0,0.000000,0.048919,0.0,0.0,0.000000,0.0,0.0,0.057918,0.0,0.0
Chunk_1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.057497,0.0,0.0
Chunk_2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
Chunk_3,0.070660,0.042522,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
Chunk_4,0.119405,0.071857,0.0,0.0,0.041625,0.0,0.0,0.000000,0.0,0.0


In [ ]:
query_exemplo = "De que forma o rio Paquequer é caracterizado na obra?"

# gera o vetor para a query de exemplo
vetor_query = chatbot.searcher.query_vector(query_exemplo)

print(f"\nQuery de exemplo: '{query_exemplo}'")
print(f"Formato do vetor da query: {vetor_query.shape}")
print("\n--- VETOR DA QUERY (TF-IDF) ---")
print(vetor_query)


Query de exemplo: 'De que forma o rio Paquequer é caracterizado na obra?'
Formato do vetor da query: (1, 1000)

--- VETOR DA QUERY (TF-IDF) ---
[[0.         0.         0.         0.         0.25181091 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.37013079 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.

In [ ]:
print(f"Buscando por: '{query_exemplo}'\n")

# Executa a busca com a query de exemplo
resultados = chatbot.search(query_exemplo, top_k=3)

print("--- TEXTOS RETORNADOS ---")
for r in resultados:
    display(HTML(f"<div style='border: 1px solid #ccc; padding: 10px; margin-bottom: 10px;'>"
                 f"<b>Score: {r['score']:.4f}</b>"
                 f"<p style='font-style: italic;'>...{r['text']}...</p>"
                 f"</div>"))


Buscando por: 'De que forma o rio Paquequer é caracterizado na obra?'

--- TEXTOS RETORNADOS ---


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

chatbot_window = widgets.VBox()
chatbot_window.layout = widgets.Layout(
    border='2px solid #4CAF50',
    padding='15px',
    margin='20px auto',
    width='90%',
    max_width='700px',
    box_shadow='2px 2px 12px rgba(0, 0, 0, 0.2)',
    background_color='#f9f9f9'
)

chatbot_title = widgets.HTML(
    value="<h3 style='color: #4CAF50; margin-top: 0;'> Chatbot - Perguntas sobre 'O Guarani'</h3>"
)

text_input = widgets.Text(
    placeholder='Digite sua pergunta sobre "O Guarani"',
    description='Pergunta:',
    layout=widgets.Layout(width='100%')
)

button = widgets.Button(
    description='Buscar Resposta',
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='30%', margin='10px 0px')
)

output_area = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        query = text_input.value.strip()
        if not query:
            print("Digite uma pergunta.")
            return

        print(f"Buscando por: '{query}'\n")
        # apenas o primeiro
        results = chatbot.search(query, top_k=1)
        intent = chatbot.analyze_intent(query)

        if not results:
            print("Nenhum resultado relevante encontrado.")
            return

        # template de resposta
        tpl = {
            'person':  'Sobre essa pessoa, encontrei:\n"{text}"',
            'place':   'Referente ao local, veja:\n"{text}"',
            'time':    'Quanto ao tempo, encontrei:\n"{text}"',
            'method':  'Sobre o método, observe:\n"{text}"',
            'reason':  'Sobre a razão, encontrei:\n"{text}"',
            'general': 'Encontrei este trecho que pode ajudar:\n"{text}"'
        }

        top_text = results[0]['text']
        qtype = intent.get('question_type', 'general')
        reply    = tpl.get(qtype, tpl['general']).format(text=top_text)

        safe_html = reply.replace("\n", "<br/>")
        display(HTML(f"""
            <div style='border-left: 4px solid #4CAF50;
                        background: #fff; padding: 10px; margin-top: 15px;'>
              <p style='margin: 0; color: #333;'>{safe_html}</p>
            </div>
        """))

button.on_click(on_button_clicked)

chatbot_window.children = [
    chatbot_title,
    text_input,
    button,
    output_area
]

display(chatbot_window)
